In [1]:
import torch
from scipy import io
from tensorflow import keras
# from tsnecuda import TSNE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
embedding = np.load('/datasets/dsc180a-wi20-public/Malware/group_data/group_02/sensitive_data/processed/node2vec/graph_2.npy')

In [3]:
g = io.loadmat('/datasets/dsc180a-wi20-public/Malware/group_data/group_02/sensitive_data/interim/graph/graph_2.mat')

In [4]:
y = g['post_label'].reshape(-1,)

In [5]:
post_cate = g['post_cate']
subreddit_cate = post_cate[:, :-1]
community_cate = post_cate[:, -1].astype(int)

In [6]:
def evaluate(clf, X_train, y_train, X_test, y_test):
    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
    ]   
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    res = {}
    for i in METRICS:
        res[i.name] = i(y_test, y_pred).numpy()
    return res

In [7]:
cate = np.array(subreddit_cate.argmax(1)).reshape(-1,)

In [8]:

neg, pos = np.bincount(y)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0
class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)
X_train, X_test, y_train, y_test = train_test_split(embedding, y, test_size=0.2)
clfs = [
        LogisticRegression(
        solver='lbfgs',verbose=False, max_iter=2000, class_weight = class_weight, n_jobs=8
        ),
        RandomForestClassifier(class_weight=class_weight, n_jobs=8)
    ]
res = {}
for clf in clfs:
    res[clf.__class__.__name__] = evaluate(clf, X_train, y_train, X_test, y_test)
res = pd.DataFrame(res).T
display(res)

Examples:
    Total: 103413
    Positive: 4409 (4.26% of total)

{0: 0.5222667770999152, 1: 11.72748922658199}


,tp,fp,tn,fn,accuracy,precision,recall,auc
LogisticRegression,551.0,8045.0,11777.0,310.0,0.596045,0.0641,0.639954,0.617046
RandomForestClassifier,0.0,0.0,19822.0,861.0,0.958372,0.0000,0.000000,0.500000


In [9]:
neg, pos = np.bincount(community_cate)
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0
class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)
X_train, X_test, y_train, y_test = train_test_split(embedding, community_cate, test_size=0.2)
clfs = [
        LogisticRegression(
        solver='lbfgs',verbose=False, max_iter=2000, class_weight = class_weight, n_jobs=8
        ),
        RandomForestClassifier(class_weight=class_weight, n_jobs=8)
    ]
res = {}
for clf in clfs:
    res[clf.__class__.__name__] = evaluate(clf, X_train, y_train, X_test, y_test)
res = pd.DataFrame(res).T
display(res)

Examples:
    Total: 103413
    Positive: 23188 (22.42% of total)

{0: 0.644518541601745, 1: 2.22988183543212}


,tp,fp,tn,fn,accuracy,precision,recall,auc
LogisticRegression,3995.0,1976.0,14026.0,686.0,0.871295,0.669067,0.853450,0.864983
RandomForestClassifier,2378.0,55.0,15947.0,2303.0,0.885993,0.977394,0.508011,0.752287


In [10]:
X_train, X_test, y_train, y_test = train_test_split(embedding, cate, test_size=0.2)
clfs = [
        LogisticRegression(
        verbose=False, max_iter=2000, class_weight = 'balanced', n_jobs=8
        ),
#         LinearSVC(class_weight=class_weight),
        RandomForestClassifier(class_weight='balanced', n_jobs=8)
    ]
res = {}
for clf in clfs:
    clf.fit(X_train, y_train)
    res[clf.__class__.__name__] = clf.score(X_test, y_test)
res

{'LogisticRegression': 0.8371609534400232,
 'RandomForestClassifier': 0.8076681332495286}